In [1]:
import requests
from bs4 import BeautifulSoup as BSoup
import pandas as pd

## Setting up for testing
Use sqlite3 to connect to the edgar_idx database

In [9]:
import sqlite3
conn3 = sqlite3.connect('edgar_idx.db')
cursor=conn3.cursor()

Set the ticker and pull out the list of 10-Q's and 10-K's for it from the database. Save each to a CSV file.

In [10]:
ticker = "MMM"
#cursor.execute('''SELECT * FROM idx WHERE Symbol=?;''', ("ABBV",))
cursor.execute('''SELECT * FROM cik_ticker_name WHERE ticker=?;''',(ticker,))
res=cursor.fetchall()
print(res)
cursor.execute('''SELECT * FROM idx WHERE cik=?;''', (res[0][0],))
recs = cursor.fetchall()
names = list(map(lambda x: x[0], cursor.description))
print(names)
df = pd.DataFrame(data=recs, columns=names)
df['date'] = pd.to_datetime(df['date'])
df.columns
df.size
df.dtypes
#print(type(recs))
print(recs)
#conn3.close()
df[df.type == "10-Q"].to_csv("TestData\\"+ticker.lower()+"_all_10qs.csv", index=None)
df[df.type == "10-K"].to_csv("TestData\\"+ticker.lower()+"_all_10ks.csv", index=None)
#df[df.type == "20-F"].to_csv("TestData\\"+ticker.lower()+"_all_20fs.csv", index=None)

[('66740', 'MMM', '3M Co')]
['cik', 'conm', 'type', 'date', 'path']
[('66740', '3M CO', '10-K', '2014-02-13', 'edgar/data/66740/0001104659-14-009773.txt'), ('66740', '3M CO', '3', '2014-01-03', 'edgar/data/66740/0001127602-14-000432.txt'), ('66740', '3M CO', '4/A', '2014-02-18', 'edgar/data/66740/0001127602-14-006865.txt'), ('66740', '3M CO', '4', '2014-01-06', 'edgar/data/66740/0001127602-14-001073.txt'), ('66740', '3M CO', '4', '2014-01-06', 'edgar/data/66740/0001127602-14-001075.txt'), ('66740', '3M CO', '4', '2014-01-06', 'edgar/data/66740/0001127602-14-001078.txt'), ('66740', '3M CO', '4', '2014-01-06', 'edgar/data/66740/0001127602-14-001081.txt'), ('66740', '3M CO', '4', '2014-01-06', 'edgar/data/66740/0001127602-14-001084.txt'), ('66740', '3M CO', '4', '2014-01-06', 'edgar/data/66740/0001127602-14-001087.txt'), ('66740', '3M CO', '4', '2014-01-06', 'edgar/data/66740/0001127602-14-001089.txt'), ('66740', '3M CO', '4', '2014-01-06', 'edgar/data/66740/0001127602-14-001091.txt'), ('

The thus created CSV files can be read back in

In [11]:
all_10Ks =pd.read_csv("TestData\\mmm_all_10ks.csv", parse_dates=['date'], dtype={'cik':str, 'conm':str, 'type':str,'path':str})
all_10Qs =pd.read_csv("TestData\\mmm_all_10qs.csv", parse_dates=['date'], dtype={'cik':str, 'conm':str, 'type':str,'path':str})

## Verifying calculated data

In [6]:
import numpy as np
import matplotlib.pyplot as plt
## NVDA
#y= np.array([3123000000, 3207000000, 2911000000, 2636000000, 2230000000]) #sales
#y= np.array([1.81, 2.05, 1.86, 1.39, 0.98]) #eps
#x=np.array([0.0, -91, -182, -273, -364])
## AAPL
#se = np.array([114949, 126878, 140199]) * 1000.0
#ni = np.array([11519, 13822, 20065 ]) * 1000000.0
#sales = np.array([53265, 61137, 88293, 52579, 45408, 52896, 78351]) * 1000000.0
#eps = np.array([2.36, 2.75, 3.92, 2.18, 1.68, 2.11, 3.38])
## ACLS
se= np.array([397074, 385614, 371527])*1000.0
ni = np.array([8838, 14669, 13915])*1000.0
sales = np.array([95374, 119333, 122185])*1000.0
eps = np.array([0.95, 2.26, 1.71])
days = np.array([0.0, -91, -182, -273, -364, -455, -546])
y = sales[:3]
x = days[:3]

In [8]:
p = np.polyfit(x[:3], y[:3], 2)
yfit = np.polyval(p, x)
sigma = (y[:3] - yfit[:3]) / y[:3]
error = sigma * sigma
res = error.sum()
plt.plot(x,y)
plt.plot(x,yfit)
plt.show()

NameError: name 'p' is not defined

In [ ]:
print(p)
print(error)
print(res)
print((sales[2] - sales[1])/(days[2] - days[1]))
print((eps[2] - eps[1])/(days[2] - days[1]))
print((y[2] - y[0])/(x[2] - x[0]))